# __Predicting Outcomes of Call Option Contracts__

# SECTION 1: Preparation

##  1.1.) Loading the python packages

In [1]:

from sklearn import datasets

# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
import random

# Load libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot
from pandas import read_csv, set_option
from pandas.plotting import scatter_matrix
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score, GridSearchCV
from evolutionary_search import EvolutionaryAlgorithmSearchCV

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
#from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier

#importing classification_report, confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix

#importing accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# import packages for hyperparameters tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

#from sklearn.metrics import balanced_accuracy_score
#from sklearn.metrics import confusion_matrix
#from imblearn.metrics import classification_report_imbalanced

#Libraries for Saving the Model
from pickle import dump
from pickle import load

import warnings
warnings.filterwarnings('ignore')



## 1.2.) Loading the Data

In [2]:
#Import call options info

df = pd.read_csv(
    Path("../Resources/test_2qs.csv")
)

#NOTE:
#use test.csv for just Q1 2022
#use test_2qs.csv for Q4 21 + Q1 22

In [3]:
df.head()

,QUOTE_READTIME,CONTRACT,SPY PRICE,EXPIRE_DATE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,C_VOLUME,C_LAST,STRIKE,STRIKE_DISTANCE_PCT,STRIKE DISTANCE,PRICECLOSE,ROI %,y,INFLATION%
0,11/29/2021 14:00,477.5 2021-12-03,466.25,2021-12-03,4.08,0.03813,0.01323,0.04436,-0.04721,0.00247,0.11638,137,0.09,477.5,0.024,11.25,0.01,-88.888889,0.0,0.9
1,11/29/2021 14:00,478.0 2021-12-03,466.25,2021-12-03,4.08,0.03425,0.01197,0.04012,-0.04335,0.00231,0.11865,1218,0.08,478.0,0.025,11.75,0.02,-75.000000,0.0,0.9
2,11/29/2021 14:00,479.0 2021-12-03,466.25,2021-12-03,4.08,0.02334,0.00877,0.02989,-0.03026,0.00188,0.11759,392,0.05,479.0,0.027,12.75,0.02,-60.000000,0.0,0.9
3,11/29/2021 14:30,462.0 2021-12-03,465.93,2021-12-03,4.06,0.66737,0.03894,0.19384,-0.36992,0.04046,0.17491,3299,6.00,462.0,0.008,-3.93,0.53,-91.166667,0.0,0.9
4,11/29/2021 14:30,462.5 2021-12-03,465.93,2021-12-03,4.06,0.64929,0.04032,0.19768,-0.37281,0.03856,0.17200,3272,5.59,462.5,0.007,-3.43,0.45,-91.949911,0.0,0.9


# SECTION 2: Exploratory Data Analysis

## 2.1.) Descriptive Statistics

In [4]:
df.shape

(3658, 20)

In [5]:
#Check for any null values and remove the null values
print('Null Values =', df.isnull().values.any())

#Drop NaNs
df.dropna()

Null Values = False


,QUOTE_READTIME,CONTRACT,SPY PRICE,EXPIRE_DATE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,C_VOLUME,C_LAST,STRIKE,STRIKE_DISTANCE_PCT,STRIKE DISTANCE,PRICECLOSE,ROI %,y,INFLATION%
0,11/29/2021 14:00,477.5 2021-12-03,466.25,2021-12-03,4.08,0.03813,0.01323,0.04436,-0.04721,0.00247,0.11638,137,0.09,477.5,0.024,11.25,0.01,-88.888889,0.0,0.9
1,11/29/2021 14:00,478.0 2021-12-03,466.25,2021-12-03,4.08,0.03425,0.01197,0.04012,-0.04335,0.00231,0.11865,1218,0.08,478.0,0.025,11.75,0.02,-75.000000,0.0,0.9
2,11/29/2021 14:00,479.0 2021-12-03,466.25,2021-12-03,4.08,0.02334,0.00877,0.02989,-0.03026,0.00188,0.11759,392,0.05,479.0,0.027,12.75,0.02,-60.000000,0.0,0.9
3,11/29/2021 14:30,462.0 2021-12-03,465.93,2021-12-03,4.06,0.66737,0.03894,0.19384,-0.36992,0.04046,0.17491,3299,6.00,462.0,0.008,-3.93,0.53,-91.166667,0.0,0.9
4,11/29/2021 14:30,462.5 2021-12-03,465.93,2021-12-03,4.06,0.64929,0.04032,0.19768,-0.37281,0.03856,0.17200,3272,5.59,462.5,0.007,-3.43,0.45,-91.949911,0.0,0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3653,2022-01-05 14:30:00,491.0 2022-01-07,473.41,1/7/2022,2.06,0.00497,0.00210,0.00546,-0.00950,-0.00016,0.16082,38,0.02,491.0,0.037,17.59,0.01,-50.000000,0.0,0.8
3654,2022-01-05 15:00:00,474.0 2022-01-07,471.69,1/7/2022,2.04,0.34400,0.06538,0.15074,-0.40846,0.01179,0.13661,9952,1.28,474.0,0.005,2.31,0.55,-57.031250,0.0,0.8
3655,2022-01-05 15:00:00,475.0 2022-01-07,471.69,1/7/2022,2.04,0.27673,0.06096,0.13782,-0.35256,0.00969,0.13330,18824,0.94,475.0,0.007,3.31,0.39,-58.510638,0.0,0.8
3656,2022-01-05 15:00:00,476.0 2022-01-07,471.69,1/7/2022,2.04,0.21498,0.05423,0.11988,-0.29226,0.00813,0.13059,22826,0.65,476.0,0.009,4.31,0.29,-55.384615,0.0,0.8


In [6]:
display(df.head())
df.tail()

,QUOTE_READTIME,CONTRACT,SPY PRICE,EXPIRE_DATE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,C_VOLUME,C_LAST,STRIKE,STRIKE_DISTANCE_PCT,STRIKE DISTANCE,PRICECLOSE,ROI %,y,INFLATION%
0,11/29/2021 14:00,477.5 2021-12-03,466.25,2021-12-03,4.08,0.03813,0.01323,0.04436,-0.04721,0.00247,0.11638,137,0.09,477.5,0.024,11.25,0.01,-88.888889,0.0,0.9
1,11/29/2021 14:00,478.0 2021-12-03,466.25,2021-12-03,4.08,0.03425,0.01197,0.04012,-0.04335,0.00231,0.11865,1218,0.08,478.0,0.025,11.75,0.02,-75.000000,0.0,0.9
2,11/29/2021 14:00,479.0 2021-12-03,466.25,2021-12-03,4.08,0.02334,0.00877,0.02989,-0.03026,0.00188,0.11759,392,0.05,479.0,0.027,12.75,0.02,-60.000000,0.0,0.9
3,11/29/2021 14:30,462.0 2021-12-03,465.93,2021-12-03,4.06,0.66737,0.03894,0.19384,-0.36992,0.04046,0.17491,3299,6.00,462.0,0.008,-3.93,0.53,-91.166667,0.0,0.9
4,11/29/2021 14:30,462.5 2021-12-03,465.93,2021-12-03,4.06,0.64929,0.04032,0.19768,-0.37281,0.03856,0.17200,3272,5.59,462.5,0.007,-3.43,0.45,-91.949911,0.0,0.9


,QUOTE_READTIME,CONTRACT,SPY PRICE,EXPIRE_DATE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,C_VOLUME,C_LAST,STRIKE,STRIKE_DISTANCE_PCT,STRIKE DISTANCE,PRICECLOSE,ROI %,y,INFLATION%
3653,2022-01-05 14:30:00,491.0 2022-01-07,473.41,1/7/2022,2.06,0.00497,0.00210,0.00546,-0.00950,-0.00016,0.16082,38,0.02,491.0,0.037,17.59,0.01,-50.000000,0.0,0.8
3654,2022-01-05 15:00:00,474.0 2022-01-07,471.69,1/7/2022,2.04,0.34400,0.06538,0.15074,-0.40846,0.01179,0.13661,9952,1.28,474.0,0.005,2.31,0.55,-57.031250,0.0,0.8
3655,2022-01-05 15:00:00,475.0 2022-01-07,471.69,1/7/2022,2.04,0.27673,0.06096,0.13782,-0.35256,0.00969,0.13330,18824,0.94,475.0,0.007,3.31,0.39,-58.510638,0.0,0.8
3656,2022-01-05 15:00:00,476.0 2022-01-07,471.69,1/7/2022,2.04,0.21498,0.05423,0.11988,-0.29226,0.00813,0.13059,22826,0.65,476.0,0.009,4.31,0.29,-55.384615,0.0,0.8
3657,2022-01-05 15:00:00,477.0 2022-01-07,471.69,1/7/2022,2.04,0.16096,0.04617,0.10052,-0.22956,0.00549,0.12919,42011,0.44,477.0,0.011,5.31,0.21,-52.272727,0.0,0.8


In [7]:
df.describe()

,SPY PRICE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,C_VOLUME,C_LAST,STRIKE,STRIKE_DISTANCE_PCT,STRIKE DISTANCE,PRICECLOSE,ROI %,y,INFLATION%
count,3658.000000,3658.000000,3658.000000,3658.000000,3658.000000,3658.000000,3658.000000,3658.000000,3658.000000,3658.000000,3658.000000,3658.000000,3658.000000,3658.000000,3658.000000,3658.000000,3658.000000
mean,468.637471,3.087868,0.294186,0.032733,0.106174,-0.202356,0.013225,0.151654,6198.571624,2.107160,474.042236,0.016970,5.404765,2.311750,7.926834,0.778568,0.839366
std,7.464809,0.801855,0.284351,0.022393,0.067390,0.147676,0.012520,0.039547,11401.515644,3.056787,8.719675,0.012228,8.069179,3.836925,109.163539,0.792472,0.048863
min,450.440000,2.000000,0.002060,0.000860,0.002780,-0.622470,-0.000350,0.089130,0.000000,0.010000,454.000000,0.000000,-15.440000,0.010000,-95.693780,0.000000,0.800000
25%,462.050000,2.210000,0.039178,0.012725,0.040195,-0.306755,0.001865,0.114358,525.500000,0.100000,468.000000,0.007000,-0.267500,0.050000,-73.096447,0.000000,0.800000
50%,468.130000,3.110000,0.186325,0.030570,0.107240,-0.200555,0.008960,0.145900,2243.000000,0.680000,475.000000,0.015000,5.580000,0.290000,-11.702630,1.000000,0.800000
75%,476.650000,4.020000,0.514508,0.049950,0.165268,-0.058397,0.022867,0.181978,6849.750000,2.897500,481.000000,0.025000,11.070000,3.030000,18.803397,1.000000,0.900000
max,479.650000,4.270000,0.957660,0.100780,0.221910,-0.003710,0.044540,0.271400,120973.000000,15.150000,491.000000,0.063000,28.560000,15.970000,476.315789,2.000000,0.900000


In [8]:
df.dtypes

QUOTE_READTIME          object
CONTRACT                object
SPY PRICE              float64
EXPIRE_DATE             object
DTE                    float64
C_DELTA                float64
C_GAMMA                float64
C_VEGA                 float64
C_THETA                float64
C_RHO                  float64
C_IV                   float64
C_VOLUME                 int64
C_LAST                 float64
STRIKE                 float64
STRIKE_DISTANCE_PCT    float64
STRIKE DISTANCE        float64
PRICECLOSE             float64
ROI %                  float64
y                      float64
INFLATION%             float64
dtype: object

## 2.2.) Feature Analysis and Exploration

### Eliminate Uncorrelated Features

In [9]:
#Calculate correlation of each feature with 'y'

correlation = df.corr()
correlation_df = abs(correlation['y'])

In [10]:
correlation_df.sort_values(ascending=False)

y                      1.000000
ROI %                  0.841607
C_IV                   0.481378
PRICECLOSE             0.401287
SPY PRICE              0.325645
STRIKE                 0.305410
C_LAST                 0.172878
C_THETA                0.146653
C_GAMMA                0.122922
C_RHO                  0.119686
C_DELTA                0.119485
STRIKE_DISTANCE_PCT    0.092887
DTE                    0.057185
C_VEGA                 0.038617
STRIKE DISTANCE        0.028777
C_VOLUME               0.014729
INFLATION%             0.012614
Name: y, dtype: float64

In [11]:
#Drop variables with less than 3% correlation with contract outcome ('y')

#drop_list_corr = sorted(list(correlation_df[correlation_df < 0.015].index))
#print(drop_list_corr)

In [12]:
#Dropping because C_IV convolutes outcome for some buggish reason

#df.drop(labels=["C_IV"], axis=1, inplace=True)

### Drop Columns Not Needed for Machine Learning

In [13]:
df.drop(columns=["QUOTE_READTIME"], inplace=True)

In [14]:
df.drop(columns=["ROI %"], inplace=True)

In [15]:
df.drop(columns=["PRICECLOSE"], inplace=True)

In [16]:
df.drop(columns=["EXPIRE_DATE"], inplace=True)

In [17]:
df.drop(columns=["CONTRACT"], inplace=True)

In [18]:
df.to_csv("../Resources/2qs_test_ready.csv", index=None)

In [ ]:
#df.to_csv("../Resources/bull_test_ready.csv", index=None)

# __*STOP HERE AND CON'T TO "forests_no_random_splits.ipynb"*__

# SECTION 3: Evaluate Algorithms and Models

## 3.1.) Train Test Split and Evaluation Metrics

In [ ]:
#Split into training and testing groups

#Split out validation dataset for the end
y = df["y"]

X = df.drop(columns='y')


In [ ]:
y.value_counts()

In [ ]:

validation_size = 0.2

seed = 7

X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=validation_size, stratify=y, random_state=seed)

In [ ]:
X_validation

In [ ]:
# Create the StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Scale both the training and testing data from the features dataset
X_train_scaled = X_scaler.transform(X_train)
X_validation_scaled = X_scaler.transform(X_validation)

## 3.2.) Quick Check of Models and Algorithms

In [ ]:
#Spot check Classification algorithms

models = []

#Boosting methods

models.append(('XGB', XGBClassifier()))
models.append(('AB', AdaBoostClassifier()))
models.append(('GBM', GradientBoostingClassifier()))

#Bagging methods

models.append(('RF', RandomForestClassifier()))
models.append(('ET', ExtraTreesClassifier()))

#SVC 
models.append(('SVC', SVC()))

In [ ]:
#Test options for classification

num_folds = 10
seed = 7

In [ ]:
results = []

names = []

scoring = 'accuracy'

for name, model in models:
    kfold = StratifiedKFold(n_splits=num_folds, random_state=seed, shuffle=True)
    cv_results = cross_val_score(model, X_train_scaled, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    findings_summary = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(findings_summary)

In [ ]:
#Compare the model outcomes

fig = pyplot.figure()
fig.suptitle('Model Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names)
fig.set_size_inches(8,4)
pyplot.show()

## 3.3.) Selecting Random Forest (or whichver tree method best), Calculating its Baseline 

In [ ]:
#Estimate accuracy on training set

rf_model = RandomForestClassifier()

cv = StratifiedKFold(n_splits=num_folds, random_state=seed, shuffle=True)
scores = cross_val_score(rf_model, X_train_scaled, y_train, scoring='accuracy', cv=cv)
avg_score = np.mean(scores)
avg_score

In [ ]:
#Estimate accuracy on validation set

rf_model = RandomForestClassifier()

rf_model.fit(X_train_scaled, y_train)

predictions = rf_model.predict(X_validation_scaled)

print(accuracy_score(y_validation, predictions))

print('Weighted Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_validation, predictions, average='weighted')))

print(classification_report(y_validation, predictions, target_names=["Strong Sell", "Sell", "Pass", "Buy", "Strong Buy", "Very High Return"]))

print(confusion_matrix(y_validation, predictions))

In [ ]:
#Visualization of the Confusion Matrix

df_matrix = pd.DataFrame(confusion_matrix(y_validation, predictions), columns=np.unique(y_validation), index = np.unique(y_validation))
df_matrix.index.name = 'Actual'
df_matrix.columns.name = 'Predicted'
sns.heatmap(df_matrix, cmap="Blues", annot=True, annot_kws={"size": 16}) 

# SECTION 4: Model Tuning 

## 4.1.) Hyperparameter Tuning for Random Forest

### APPROACH (A) 
## __*(CAUTION TAKES A LONG TIME!)*__

In [ ]:


# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {}

paramgrid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,}
              # 'min_samples_split': min_samples_split,
               #'min_samples_leaf': min_samples_leaf,
              # 'bootstrap': bootstrap}

random.seed(1)

cv = EvolutionaryAlgorithmSearchCV(estimator=RandomForestClassifier(),
                                   params=paramgrid,
                                   scoring="accuracy",
                                   cv=StratifiedKFold(n_splits=4),
                                   verbose=1,
                                   population_size=50,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=5,
                                   n_jobs=4)
cv.fit(X_train_scaled, y_train)

### APPROACH (B)
## __*(CAUTION TAKES A LONG TIME!)*__

In [ ]:
# Grid Search: (select model) Tuning

n_estimators = [20, 100, 180, 1000]
max_features = [1, 2, 3, 4]

param_grid = dict(n_estimators=n_estimators, max_features=max_features)

rf_model = RandomForestClassifier()

kfold = StratifiedKFold(n_splits=num_folds, random_state=seed, shuffle=True)

grid = GridSearchCV(estimator=rf_model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(X_train_scaled, y_train)

#Print Results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
ranks = grid_result.cv_results_['rank_test_score']
for mean, stdev, param, rank in zip(means, stds, params, ranks):
    print("#%d %f (%f) with: %r" % (rank, mean, stdev, param))

### PROCEEDING WITH BEST APPROACH

In [ ]:
#Prepare the model with parameters decided in previous cell

rf_model = RandomForestClassifier() #(n_estimators=200, max_features="sqrt", max_depth=100, min_samples_split=5, min_samples_leaf=1, bootstrap=False)
rf_model.fit(X_train_scaled, y_train)

In [ ]:
#Score predictions of training set

training_predictions = rf_model.predict(X_train_scaled)

print(accuracy_score(y_train, training_predictions))

print('Micro Precision: {:.2f}'.format(precision_score(y_train, training_predictions, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_train, training_predictions, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_train, training_predictions, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_train, training_predictions, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_train, training_predictions, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_train, training_predictions, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_train, training_predictions, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_train, training_predictions, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_train, training_predictions, average='weighted')))


print(classification_report(y_train, training_predictions, target_names=["Strong Sell", "Sell", "Pass", "Buy", "Strong Buy", "Very High Return"]))

In [ ]:
#Score predictions of validation set

predictions = rf_model.predict(X_validation_scaled)

print(accuracy_score(y_validation, predictions))

print('Micro Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_validation, predictions, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_validation, predictions, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_validation, predictions, average='weighted')))


print(classification_report(y_validation, predictions, target_names=["Strong Sell", "Sell", "Pass", "Buy", "Strong Buy", "Very High Return"]))

print(confusion_matrix(y_validation, predictions))

In [ ]:
#Visualization of the Confusion Matrix

df_matrix = pd.DataFrame(confusion_matrix(y_validation, predictions), columns=np.unique(y_validation), index = np.unique(y_validation))
df_matrix.index.name = 'Actual'
df_matrix.columns.name = 'Predicted'
sns.heatmap(df_matrix, cmap="Blues", annot=True, annot_kws={"size": 16}) 

## 4.2.) Determining Feature Importance

In [ ]:
#Feature importances

importances = rf_model.feature_importances_

In [ ]:
important_features = zip(X.columns, rf_model.feature_importances_)

In [ ]:
importances_df = pd.DataFrame(important_features)

In [ ]:
importances_df = importances_df.rename(columns={0: 'Feature', 1: 'Importance'})

In [ ]:
importances_df = importances_df.set_index('Feature')

In [ ]:
importances_df = importances_df.sort_values(by='Importance', ascending=False)

In [ ]:
importances_df

In [ ]:
#Plot the top 10 most important features

importances_df[0:10].plot(
    kind='barh',
    color='green',
    title='Feature Importance',
    legend=True)

## 4.2. Bayesian Optimization with HYPEROPT for XGB

## __*(CAUTION TAKES A COUPLE OF MIN!)*__

In [ ]:
#Initialize domain space for range of values 
 
space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1, 9),
        'reg_alpha' : hp.quniform('reg_alpha', 40, 180, 1),
        'reg_lambda' : hp.uniform('reg_lambda', 0, 1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5, 1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0
    }


The available hyperopt optimization algorithms are -

hp.choice(label, options) — Returns one of the options, which should be a list or tuple.

hp.randint(label, upper) — Returns a random integer between the range [0, upper).

hp.uniform(label, low, high) — Returns a value uniformly between low and high.

hp.quniform(label, low, high, q) — Returns a value round(uniform(low, high) / q) * q, i.e it rounds the decimal values and returns an integer.

hp.normal(label, mean, std) — Returns a real value that’s normally-distributed with mean and standard deviation sigma.

In [ ]:
def objective(space):
    xgb_model = xgb.XGBClassifier(
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [(X_train_scaled, y_train), (X_validation_scaled, y_validation)]
    
    xgb_model.fit(X_train_scaled, y_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    

    pred = xgb_model.predict(X_validation_scaled)
    accuracy = accuracy_score(y_validation, pred)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }

In [ ]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

In [ ]:
print("The best hyperparameters are : ","\n")
print(best_hyperparams)

In [ ]:
#Establish XGB model instance
#Fitting training dataset to model

xgb_model = xgb.XGBClassifier(colsample_bytree=0.7417632345631163, max_depth=8, gamma=1.5654068039190379, min_child_weight=3.0, reg_lambda=0.8585451920481999, reg_alpha=0)
xgb_model.fit(X_train_scaled, y_train)

In [ ]:
#Predicting on training set

training_predictions = xgb_model.predict(X_train_scaled)

print(accuracy_score(y_train, training_predictions))

print('Weighted Precision: {:.2f}'.format(precision_score(y_train, training_predictions, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_train, training_predictions, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_train, training_predictions, average='weighted')))

print(classification_report(y_train, training_predictions, target_names=["Strong Sell", "Sell", "Pass", "Buy", "Strong Buy", "Very High Return"]))

In [ ]:
#Predicting on validation set

predictions = xgb_model.predict(X_validation_scaled)

print(accuracy_score(y_validation, predictions))

print('Weighted Precision: {:.2f}'.format(precision_score(y_validation, predictions, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_validation, predictions, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_validation, predictions, average='weighted')))

print(classification_report(y_validation, predictions, target_names=["Strong Sell", "Sell", "Pass", "Buy", "Strong Buy", "Very High Return"]))

print(confusion_matrix(y_validation, predictions))

In [ ]:
#Get probabilities of predictions

pred_proba = xgb_model.predict_proba(X_validation_scaled)
pred_proba

In [ ]:
proba_df = pd.DataFrame(pred_proba.round(2))

In [ ]:
#Formatting probabilities DataFrame

proba_df["Actual"] = y_validation.reset_index(drop=True)
proba_df.columns = ["'Strong Sell' Probability", "'Sell' Probability", "'Pass' Probability", "'Buy' Probability", "'Strong Buy' Probability", "'Very High Return' Prob.", "Actual"]
proba_df

In [ ]:
#Labeling caterogires

proba_df.loc[proba_df["Actual"] == 0, "Actual"] = "Strong Sell"
proba_df.loc[proba_df["Actual"] == 1, "Actual"] = "Sell"
proba_df.loc[proba_df["Actual"] == 2, "Actual"] = "Pass"
proba_df.loc[proba_df["Actual"] == 3, "Actual"] = "Buy"
proba_df.loc[proba_df["Actual"] == 4, "Actual"] = "Strong Buy"
proba_df.loc[proba_df["Actual"] == 5, "Actual"] = "Very High Return"

proba_df.head(50)

In [ ]:
#Get ROC AUC score

roc_auc_score(y_validation, pred_proba, multi_class="ovr")

## 4.2.) Determining Feature Importance for XGBoost Model

In [ ]:
#Feature importances

importances = xgb_model.feature_importances_

In [ ]:
important_features = zip(X.columns, xgb_model.feature_importances_)

In [ ]:
importances_df = pd.DataFrame(important_features)

In [ ]:
importances_df = importances_df.rename(columns={0: 'Feature', 1: 'Importance'})

In [ ]:
importances_df = importances_df.set_index('Feature')

In [ ]:
importances_df = importances_df.sort_values(by='Importance', ascending=False)

In [ ]:
importances_df

In [ ]:
#Plot the top 10 most important features

importances_df[0:10].plot(
    kind='barh',
    color='green',
    title='Feature Importance',
    legend=True)

In [ ]:
#et_model = ExtraTreesClassifier()
#et_model.fit(X_train_scaled, y_train)

In [ ]:
#ab_model = AdaBoostClassifier()
#ab_model.fit(X_train_scaled, y_train)

In [ ]:
#gb_model = GradientBoostingClassifier()
#ab_model.fit(X_train_scaled, y_train)